<a href="https://colab.research.google.com/github/DinalAdithya/Learning-PYTorch/blob/main/PyTorch_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PyTorch Workflow
topics = {1: "Data (prepare and load)",
          2: "Build model",
          3: "Fitting the model to data (tranning)",
          4: "Making prediction and evaluvating model(inference)",
          5: "Saving and loading model",
          6: "Putting it all together"}


In [4]:
import torch
from torch import nn # nn contains all of pytorch's building blocks for neural networks
import numpy as np
import matplotlib.pyplot as plt

torch.__version__

'2.1.0+cu121'

#Data (prepare and load)

Machine learning inlude 2 parts

1.   Get data into a numerical representation.
2.   Buil a model to learn patterns in taht numerical representation



In [5]:
# create known parameters
weight = 0.7
bias   = 0.3

# Create
start = 0
end = 1
step = 0.02
x = torch.arange(start, end, step).unsqueeze( dim=1 )
y = weight * x + bias

x[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [6]:
len(x),len(y)

(50, 50)